In [3]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

import glob

latents_dir = "/data/PycharmProjects/cytof_benchmark/results/latent_data/"
latent_files = glob.glob(latents_dir + "*/*/*/test.csv")

dataset_dirs = {
    'OrganoidDataset':'/data/PycharmProjects/cytof_benchmark/data/organoids',
    'CafDataset':'/data/PycharmProjects/cytof_benchmark/data/caf',
    'ChallengeDataset':'/data/PycharmProjects/cytof_benchmark/data/breast_cancer_challenge',
}

latents_list = list()
for latent_file in latent_files:
    dataset = latent_file.split('/')[-2]
    model = latent_file.split('/')[-3]
    dim = latent_file.split('/')[-4]
    latents_list.append((dataset,model,dim,latent_file))
latents_list

[('CafDataset',
  'HyperSphericalVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/CafDataset/test.csv'),
 ('ChallengeDataset',
  'HyperSphericalVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/ChallengeDataset/test.csv'),
 ('OrganoidDataset',
  'HyperSphericalVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/OrganoidDataset/test.csv'),
 ('CafDataset',
  'DBetaVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/DBetaVAE/CafDataset/test.csv'),
 ('ChallengeDataset',
  'DBetaVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/DBetaVAE/ChallengeDataset/test.csv'),
 ('OrganoidDataset',
  'DBetaVAE',
  'dim5',
  '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/DBetaVAE/OrganoidDataset/test.csv'),
 ('CafDataset',
  'WAE_MMD',
  'dim5',
  '/data/PycharmProjects/cytof_benc

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import datasets

metadata_dict = dict()
save_dir = Path('/data/PycharmProjects/cytof_benchmark/results/latent_plots')

for dataset_name,model_name,dim_name,latent_path in tqdm(latents_list):
    if dataset_name not in metadata_dict:
        dataset_class = getattr(datasets, dataset_name)
        dataset_dir = dataset_dirs[dataset_name]

        dataset = dataset_class(data_dir=dataset_dir)
        metadata_dict[dataset_name] = dataset.test[1]

    y_test = metadata_dict[dataset_name]
    latent_df = pd.read_csv(latent_path,index_col=0)
    latent_dim = int(dim_name[-1])

    for variable_name in y_test.columns:

        plot_data = pd.concat([latent_df, y_test], axis=1).head(100000)[['VAE1','VAE2',variable_name]]
        sns.set(rc={'figure.figsize':(12,12)})
        dot_plot = sns.scatterplot(x="VAE1", y="VAE2",
                                   data=plot_data,
                                   hue=variable_name,
                                   legend=True,
                                   alpha=0.5)
        plot_file = save_dir / dim_name / model_name / dataset_name / (variable_name +'_VAE12.png')
        plot_file.parent.mkdir(parents=True, exist_ok=True)
        if latent_dim==2:
            plot_data.to_csv(save_dir / dim_name / model_name / dataset_name / (variable_name +'_plot.csv'),index=None)
        plt.savefig(plot_file)
        plt.clf()

        # UMAP plot
    if latent_dim > 2 or model_name == 'HyperSphericalVAE':
        import umap
        import umap.plot
        from bokeh.io import save
        umap_points = 100000
        u = pd.DataFrame(umap.UMAP(low_memory=False, n_jobs=8).fit_transform(latent_df.head(umap_points)),columns=['UMAP1','UMAP2'])
        for variable_name in y_test.columns:
            umap_data = pd.concat([u, y_test], axis=1).head(umap_points)[['UMAP1','UMAP2',variable_name]]
            sns.set(rc={'figure.figsize':(12,12)})
            dot_plot = sns.scatterplot(x='UMAP1', y='UMAP2',
                                   data=umap_data,
                                   hue=variable_name,
                                   legend=True,
                                   alpha=0.5)
            plt.savefig(save_dir / dim_name / model_name / dataset_name / (variable_name +'_umap.png'))
            if not (model_name == 'HyperSphericalVAE' and latent_dim == 2):
                umap_data.to_csv(save_dir / dim_name / model_name / dataset_name / (variable_name +'_plot.csv'),index=None)
            plt.clf()

    # Spherical plots
    if model_name == 'HyperSphericalVAE' and latent_dim == 2:
        import cartopy
        import geopandas
        from shapely.geometry import Point
        import plotly.express as px

        sphere = geopandas.GeoSeries(latent_df.apply(Point, axis=1), crs=cartopy.crs.Geocentric())
        projected = sphere.to_crs(cartopy.crs.Mollweide())
        for variable_name in y_test.columns:
            data = pd.DataFrame({'x': projected.x,
                                'y': projected.y,
                                variable_name: y_test[variable_name]
                                 })

            # 2d projection plot
            sns.scatterplot(x="x", y="y",
                    data=data.head(10000),
                    hue=variable_name,  # color by cluster
                    legend=True,
                    alpha=0.5)
            plt.axis('equal')
            plt.savefig(save_dir / dim_name / model_name / dataset_name / (variable_name +'_projected.png'))
            data.to_csv(save_dir / dim_name / model_name / dataset_name / (variable_name +'_plot.csv'))
            plt.clf()
            # 3d point cloud plot
            plot_data = pd.concat([latent_df, y_test], axis=1).head(10000)
            fig = px.scatter_3d(plot_data, x='VAE1', y='VAE2', z='VAE3', color=variable_name)
            fig.update_traces(marker_size=2)
            fig.write_html(save_dir / dim_name / model_name / dataset_name / (variable_name +'_point-cloud.html'))

 33%|███▎      | 12/36 [19:42<33:05, 82.71s/it]  /data/PycharmProjects/cytof_benchmark/venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
 36%|███▌      | 13/36 [32:30<1:51:18, 290.38s/it]/data/PycharmProjects/cytof_benchmark/venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.

 39%|███▉      | 14/36 [37:05<1:44:48, 285.84s/it]/data/PycharmProjects/cytof_benchmark/venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.

100%|██████████| 36/36 [1:00:39<

<Figure size 1200x1200 with 0 Axes>